In [99]:
import numpy as np 
from keras.models import Model
from keras.layers import TimeDistributed,Conv1D,Dense,Embedding,Input,Dropout,LSTM,Bidirectional,MaxPooling1D,Flatten,concatenate
from keras.utils import Progbar
from keras.preprocessing.sequence import pad_sequences
from keras.initializers import RandomUniform

epochs = 50

def tag_dataset(dataset):
    correctLabels = []
    predLabels = []
    b = Progbar(len(dataset))
    for i,data in enumerate(dataset):    
        tokens, casing,char, labels = data
        tokens = np.asarray([tokens])     
        casing = np.asarray([casing])
        char = np.asarray([char])
        pred = model.predict([tokens, casing,char], verbose=False)[0]   
        pred = pred.argmax(axis=-1) #Predict the classes            
        correctLabels.append(labels)
        predLabels.append(pred)
        b.update(i)
    return predLabels, correctLabels


In [100]:
import pandas as pd
df = pd.read_csv('../data/sentences1.csv', header=None, delimiter=';', error_bad_lines=False, names=['website','text', 'keywords'])
df.reindex(np.random.permutation(df.index))

df = df[df['website'] != 'test']
df

,website,text,keywords
0,securelist,Analysis of the malicious CCleaner code allowe...,"['APT17', 'Aurora']"
1,securelist,"The Platinum, APT16, EvilPost and SPIVY groups...","['Platinum', 'APT16', 'EvilPost', 'SPIVY']"
2,securelist,This variant was also used by the APT16 group ...,"['APT16', 'ELMER', 'backdoor']"
3,securelist,"The CnC has been used in other APT incidents, ...","['admin@338', 'Temper', 'Panda']"
4,securelist,The TwoForOne (Platinum) group is described in...,"['APT16', 'EvilPost', 'Danti']"
5,securelist,The Mandiant reports starts off by stating tha...,"['APT1', 'Chinese']"
6,securelist,BlackOasis is an APT group we have been tracki...,"['BlackOasis', 'APT']"
7,securelist,"In its affidavit for sink-holing the C2, the F...","['Sofacy', 'APT28', 'Pawn', 'Storm', 'Sednit',..."
8,securelist,"Sofacy Group, also known as apt28, sandworm, x...","['Sofacy', 'apt28', 'sandworm', 'x-agent', 'pa..."
9,securelist,"Sofacy (aka APT28, Fancy Bear and Tsar Team) i...","['Sofacy', 'APT28', 'Fancy', 'Bear', 'Tsar', '..."


In [101]:


from nltk.corpus import stopwords
import re
from ast import literal_eval
stop = stopwords.words('english')

def normalize_text(text):
    #norm_text = text.lower()
    norm_text = text
    # Replace breaks with spaces
    norm_text = norm_text.replace('<br />', '')
    # Pad punctuation with spaces on both sides
    norm_text = re.sub(r"([\.\",\(\)!\?;:])", "\\1", norm_text)
    norm_text = ' '.join(norm_text.split())
    return norm_text

def remove_stop_words(text):
    return " ".join([item.lower() for item in text.split() if item not in stop])

def remove_non_ascii(text):
    return ''.join(["" if ord(i) < 32 or ord(i) > 126 else i for i in text])

df['text'] = df['text'].apply(remove_non_ascii)
df['text'] = df['text'].apply(normalize_text)
#df['text'] = df['text'].apply(remove_stop_words)
df["text"] = df['text'].str.replace('[^\w\s]','')
df['keywords'] = df['keywords'].apply(literal_eval)

In [102]:
df

,website,text,keywords
0,securelist,Analysis of the malicious CCleaner code allowe...,"[APT17, Aurora]"
1,securelist,The Platinum APT16 EvilPost and SPIVY groups w...,"[Platinum, APT16, EvilPost, SPIVY]"
2,securelist,This variant was also used by the APT16 group ...,"[APT16, ELMER, backdoor]"
3,securelist,The CnC has been used in other APT incidents a...,"[admin@338, Temper, Panda]"
4,securelist,The TwoForOne Platinum group is described in M...,"[APT16, EvilPost, Danti]"
5,securelist,The Mandiant reports starts off by stating tha...,"[APT1, Chinese]"
6,securelist,BlackOasis is an APT group we have been tracki...,"[BlackOasis, APT]"
7,securelist,In its affidavit for sinkholing the C2 the FBI...,"[Sofacy, APT28, Pawn, Storm, Sednit, STRONTIUM..."
8,securelist,Sofacy Group also known as apt28 sandworm xage...,"[Sofacy, apt28, sandworm, x-agent, pawn, storm..."
9,securelist,Sofacy aka APT28 Fancy Bear and Tsar Team is a...,"[Sofacy, APT28, Fancy, Bear, Tsar, Team]"


In [103]:
result = []
for index, row in df.iterrows():
    sentence = row.text
    s = set(map(lambda x: x.lower(), row.keywords))
    text = []
    for word in sentence.split(' '):
        word = word.strip()
        if word != '' and word.lower() in s:
            label = "GROUP_NAME"
        else:
            label = "O"
        text.append([word, label])
    result.append(text)

In [104]:
result

[[['Analysis', 'O'],
  ['of', 'O'],
  ['the', 'O'],
  ['malicious', 'O'],
  ['CCleaner', 'O'],
  ['code', 'O'],
  ['allowed', 'O'],
  ['us', 'O'],
  ['to', 'O'],
  ['correlate', 'O'],
  ['it', 'O'],
  ['with', 'O'],
  ['a', 'O'],
  ['couple', 'O'],
  ['of', 'O'],
  ['other', 'O'],
  ['backdoors', 'O'],
  ['that', 'O'],
  ['are', 'O'],
  ['known', 'O'],
  ['to', 'O'],
  ['have', 'O'],
  ['been', 'O'],
  ['used', 'O'],
  ['in', 'O'],
  ['the', 'O'],
  ['past', 'O'],
  ['by', 'O'],
  ['APT', 'O'],
  ['groups', 'O'],
  ['from', 'O'],
  ['the', 'O'],
  ['Axiom', 'O'],
  ['umbrella', 'O'],
  ['such', 'O'],
  ['as', 'O'],
  ['APT17', 'GROUP_NAME'],
  ['also', 'O'],
  ['known', 'O'],
  ['as', 'O'],
  ['Aurora', 'GROUP_NAME']],
 [['The', 'O'],
  ['Platinum', 'GROUP_NAME'],
  ['APT16', 'GROUP_NAME'],
  ['EvilPost', 'GROUP_NAME'],
  ['and', 'O'],
  ['SPIVY', 'GROUP_NAME'],
  ['groups', 'O'],
  ['were', 'O'],
  ['already', 'O'],
  ['known', 'O'],
  ['to', 'O'],
  ['use', 'O'],
  ['this', 'O'],
  [

In [105]:
def addCharInformatioin(Sentences):
    for i,sentence in enumerate(Sentences):
        for j,data in enumerate(sentence):
            chars = [c for c in data[0]]
            Sentences[i][j] = [data[0],chars,data[1]]
    return Sentences

In [106]:
result = addCharInformatioin(result)

In [107]:
trainSentences = result[:int(len(result) * .8)]
testSentences = result[int(len(result) * .8):]

In [108]:
labelSet = set()
words = {}

for dataset in [trainSentences, testSentences]:
    for sentence in dataset:
        for token,char,label in sentence:
            labelSet.add(label)
            words[token.lower()] = True

label2Idx = {}
for label in labelSet:
    label2Idx[label] = len(label2Idx)

In [109]:
case2Idx = {'numeric': 0, 'allLower':1, 'allUpper':2, 'initialUpper':3, 'other':4, 'mainly_numeric':5, 'contains_digit': 6, 'PADDING_TOKEN':7}
caseEmbeddings = np.identity(len(case2Idx), dtype='float32')

In [110]:
word2Idx = {}
wordEmbeddings = []

# fEmbeddings = open("glove.6B.100d.txt")
fEmbeddings = open("word2vec.txt")

for line in fEmbeddings:
    split = line.strip().split(" ")
    word = split[0]
    
    if len(word2Idx) == 0: #Add padding+unknown
        word2Idx["PADDING_TOKEN"] = len(word2Idx)
        vector = np.zeros(len(split)-1) #Zero vector vor 'PADDING' word
        wordEmbeddings.append(vector)
        
        word2Idx["UNKNOWN_TOKEN"] = len(word2Idx)
        vector = np.random.uniform(-0.25, 0.25, len(split)-1)
        wordEmbeddings.append(vector)

    if split[0].lower() in words:
        vector = np.array([float(num) for num in split[1:]])
        wordEmbeddings.append(vector)
        word2Idx[split[0]] = len(word2Idx)
        
wordEmbeddings = np.array(wordEmbeddings)

char2Idx = {"PADDING":0, "UNKNOWN":1}
for c in " 0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ.,-_()[]{}!?:;#'\"/\\%$`&=*+@^~|":
    char2Idx[c] = len(char2Idx)

In [111]:
wordEmbeddings

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.04564428,  0.21211099, -0.06987944, ..., -0.24225977,
        -0.24161206, -0.01009185],
       [ 0.26070642, -1.1021085 , -0.2649278 , ..., -0.00195244,
         0.97385913, -0.30661538],
       ...,
       [ 0.03101471, -0.06764489, -0.02508294, ..., -0.01494516,
         0.0385602 , -0.01278899],
       [ 0.0341464 , -0.07367665, -0.02550976, ..., -0.01468766,
         0.04396356, -0.01185723],
       [ 0.02275341, -0.04528004, -0.01716731, ..., -0.01916434,
         0.01984791, -0.01104459]])

In [119]:
def createMatrices(sentences, word2Idx, label2Idx, case2Idx,char2Idx):
    unknownIdx = word2Idx['UNKNOWN_TOKEN']
    paddingIdx = word2Idx['PADDING_TOKEN']    
        
    dataset = []
    
    wordCount = 0
    unknownWordCount = 0
    
    for sentence in sentences:
        wordIndices = []    
        caseIndices = []
        charIndices = []
        labelIndices = []
        
        for word,char,label in sentence:  
            wordCount += 1
            if word in word2Idx:
                wordIdx = word2Idx[word]
            elif word.lower() in word2Idx:
                wordIdx = word2Idx[word.lower()]                 
            else:
                wordIdx = unknownIdx
                unknownWordCount += 1
            charIdx = []
            for x in char:
                charIdx.append(char2Idx[x])
            #Get the label and map to int            
            wordIndices.append(wordIdx)
            caseIndices.append(getCasing(word, case2Idx))
            charIndices.append(charIdx)
            labelIndices.append(label2Idx[label])
        dataset.append([wordIndices, caseIndices, charIndices, labelIndices]) 
        
    return dataset

def padding(Sentences):
    maxlen = 52
    for sentence in Sentences:
        char = sentence[2]
        for x in char:
            maxlen = max(maxlen,len(x))
    for i,sentence in enumerate(Sentences):
        Sentences[i][2] = pad_sequences(Sentences[i][2],52,padding='post')
    return Sentences

def getCasing(word, caseLookup):   
    casing = 'other'
    
    numDigits = 0
    for char in word:
        if char.isdigit():
            numDigits += 1
            
    digitFraction = numDigits / float(len(word))
    
    if word.isdigit():
        casing = 'numeric'
    elif digitFraction > 0.5:
        casing = 'mainly_numeric'
    elif word.islower():
        casing = 'allLower'
    elif word.isupper():
        casing = 'allUpper'
    elif word[0].isupper():
        casing = 'initialUpper'
    elif numDigits > 0:
        casing = 'contains_digit'
    return caseLookup[casing]

train_set = padding(createMatrices(trainSentences,word2Idx,  label2Idx, case2Idx,char2Idx))
test_set = padding(createMatrices(testSentences, word2Idx, label2Idx, case2Idx,char2Idx))

idx2Label = {v: k for k, v in label2Idx.items()}

In [121]:
# train_set[0][3]
# char2Idx

In [78]:
def createBatches(data):
    l = []
    for i in data:
        l.append(len(i[0]))
    l = set(l)
    batches = []
    batch_len = []
    z = 0
    for i in l:
        for batch in data:
            if len(batch[0]) == i:
                batches.append(batch)
                z += 1
        batch_len.append(z)
    return batches,batch_len

train_batch,train_batch_len = createBatches(train_set)
test_batch,test_batch_len = createBatches(test_set)

In [131]:
words_input = Input(shape=(None,),dtype='int32',name='words_input')

words = Embedding(input_dim=wordEmbeddings.shape[0], output_dim=100,  weights=[wordEmbeddings], trainable=False)(words_input)

casing_input = Input(shape=(None,), dtype='int32', name='casing_input')
casing = Embedding(output_dim=caseEmbeddings.shape[1], input_dim=caseEmbeddings.shape[0], weights=[caseEmbeddings], trainable=False)(casing_input)

character_input=Input(shape=(None,52,),name='char_input')
embed_char_out=TimeDistributed(Embedding(len(char2Idx),30,embeddings_initializer=RandomUniform(minval=-0.5, maxval=0.5)), name='char_embedding')(character_input)

dropout= Dropout(0.5)(embed_char_out)

conv1d_out= TimeDistributed(Conv1D(kernel_size=3, filters=30, padding='same',activation='tanh', strides=1))(dropout)

maxpool_out=TimeDistributed(MaxPooling1D(52))(conv1d_out)

char = TimeDistributed(Flatten())(maxpool_out)
char = Dropout(0.5)(char)

output = concatenate([words, casing,char])
output = Bidirectional(LSTM(20, return_sequences=True, dropout=0.50, recurrent_dropout=0.25))(output)
output = TimeDistributed(Dense(len(label2Idx), activation='softmax'))(output)

model = Model(inputs=[words_input, casing_input,character_input], outputs=[output])
model.compile(loss='sparse_categorical_crossentropy', optimizer='nadam')
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
char_input (InputLayer)         (None, None, 52)     0                                            
__________________________________________________________________________________________________
char_embedding (TimeDistributed (None, None, 52, 30) 2850        char_input[0][0]                 
__________________________________________________________________________________________________
dropout_15 (Dropout)            (None, None, 52, 30) 0           char_embedding[0][0]             
__________________________________________________________________________________________________
time_distributed_29 (TimeDistri (None, None, 52, 30) 2730        dropout_15[0][0]                 
__________________________________________________________________________________________________
time_distr

In [132]:
def iterate_minibatches(dataset,batch_len): 
    start = 0
    for i in batch_len:
        tokens = []
        caseing = []
        char = []
        labels = []
        data = dataset[start:i]
        start = i
        for dt in data:
            t,c,ch,l = dt
            l = np.expand_dims(l,-1)
            tokens.append(t)
            caseing.append(c)
            char.append(ch)
            labels.append(l)
        yield np.asarray(labels),np.asarray(tokens),np.asarray(caseing),np.asarray(char)

for epoch in range(epochs):    
    print("Epoch %d/%d"%(epoch,epochs))
    a = Progbar(len(train_batch_len))
    for i,batch in enumerate(iterate_minibatches(train_batch,train_batch_len)):
        labels, tokens, casing,char = batch       
        model.train_on_batch([tokens, casing,char], labels)
        a.update(i)
    print(' ')

Epoch 0/50
13/16 [=======================>......] - ETA: 1s 
Epoch 1/50
15/16 [===========================>..] - ETA: 0s 
Epoch 2/50
13/16 [=======================>......] - ETA: 0s 
Epoch 3/50
15/16 [===========================>..] - ETA: 0s 
Epoch 4/50
13/16 [=======================>......] - ETA: 0s 
Epoch 5/50
15/16 [===========================>..] - ETA: 0s 
Epoch 6/50
13/16 [=======================>......] - ETA: 0s 
Epoch 7/50
15/16 [===========================>..] - ETA: 0s 
Epoch 8/50
15/16 [===========================>..] - ETA: 0s 
Epoch 9/50
13/16 [=======================>......] - ETA: 0s 
Epoch 10/50
13/16 [=======================>......] - ETA: 0s 
Epoch 11/50
15/16 [===========================>..] - ETA: 0s 
Epoch 12/50
14/16 [=========================>....] - ETA: 0s 
Epoch 13/50
15/16 [===========================>..] - ETA: 0s 
Epoch 14/50
15/16 [===========================>..] - ETA: 0s 
Epoch 15/50
13/16 [=======================>......] - ETA: 0s 
Epoch 16/50
15/16 

In [133]:
def compute_f1(predictions, correct, idx2Label): 
    label_pred = []    
    for sentence in predictions:
        label_pred.append([idx2Label[element] for element in sentence])
        
    label_correct = []    
    for sentence in correct:
        label_correct.append([idx2Label[element] for element in sentence])
            
    
    prec = compute_precision(label_pred, label_correct)
    rec = compute_precision(label_correct, label_pred)
    
    
    f1 = 0
    if (rec+prec) > 0:
        f1 = 2.0 * prec * rec / (prec + rec);
        
    return prec, rec, f1

def compute_precision(guessed_sentences, correct_sentences):
    assert(len(guessed_sentences) == len(correct_sentences))
    correctCount = 0
    count = 0
    
    for sentenceIdx in range(len(guessed_sentences)):
        guessed = guessed_sentences[sentenceIdx]
        correct = correct_sentences[sentenceIdx]
        idx = 0
        while idx < len(guessed):    
            count += 1
            if guessed[idx] == correct[idx]:
                idx += 1
                correctlyFound = True
                correctCount += 1
                while idx < len(guessed) and guessed[idx][0] == 'I': #Scan until it no longer starts with I
                    if guessed[idx] != correct[idx]:
                        correctlyFound = False

                    idx += 1
            else:
                idx += 1
    
    precision = 0
    if count > 0:    
        precision = float(correctCount) / count
        
    return precision

predLabels, correctLabels = tag_dataset(test_batch)        
pre_test, rec_test, f1_test= compute_f1(predLabels, correctLabels, idx2Label)

0/5 [..............................] - ETA: 0s

In [134]:
print("Test-Data: Prec: %.3f, Rec: %.3f, F1: %.3f" % (pre_test, rec_test, f1_test))

Test-Data: Prec: 0.961, Rec: 0.961, F1: 0.961
